In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [1]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import gdata_dir, scratch_dir
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.topography import pysheds_accumulation, catchment_gullies, catchment_ridges, calculate_slope

Downloaded _VPeff_2021.nc


In [2]:
# Choosing location
lat, lon = -34.3890427, 148.469499
buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total
stub = "MILG_1km"
start_year = 2020
end_year = 2021

# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = os.path.join(scratch_dir, "tmp")

In [4]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_terrain_original.tif
Saved /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_terrain_cleaned.tif
CPU times: user 113 ms, sys: 17 ms, total: 130 ms
Wall time: 252 ms


In [6]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable
# variables = ['Clay', 'Silt', 'Sand', 'pH_CaCl2', 'Bulk_Density', 'Available_Water_Capacity', 'Effective_Cation_Exchange_Capacity', 'Total_Nitrogen', 'Total_Phosphorus']
variables = ['Clay', 'Sand']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Downloaded /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Clay.tif
Downloaded /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Sand.tif
CPU times: user 98.5 ms, sys: 8.06 ms, total: 107 ms
Wall time: 14.2 s


In [8]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Maximum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Minimum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_Annual_Rainfall_2020_2021_average.tif
CPU times: user 179 ms, sys: 30.6 ms, total: 210 ms
Wall time: 1.09 s


In [11]:
%%time
# Download modelled variables from OzWald
variables = ['SN', 'Ssoil']
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_SN_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_SN_2021.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_Ssoil_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_1km_Ssoil_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_1km_ozwald_8day.nc
CPU times: user 324 ms, sys: 83.7 ms, total: 408 ms
Wall time: 4.44 s


In [14]:
%%time
variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, 0.1, start_year, end_year, outdir, "MILG_20km", tmp_dir)

Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_VPeff_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_VPeff_2021.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_Uavg_2020.nc
Downloaded /scratch/xe2/cb8590/tmp/MILG_20km_Uavg_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/MILG_20km_ozwald_daily.nc
CPU times: user 121 ms, sys: 14.4 ms, total: 136 ms
Wall time: 1.2 s
